In [1]:
import sqlite3
from matplotlib import pyplot as plt
# import cv2

In [2]:
from pykafka import KafkaClient

# client = KafkaClient(hosts="192.168.1.111:9092")
def send_profile_data(data):
        client = KafkaClient(hosts="2.tcp.ngrok.io:12218")
        topic = client.topics['test']
        producer = topic.get_sync_producer()
        producer.produce(data)

In [3]:
def getProfile(emp_id) :
                conn = sqlite3.connect("../../dbfs/FileStore/tables/FR/dev.db")
                # cur = conn.cursor()
                cmd = "SELECT * FROM EMP WHERE emp_id=" + str(emp_id)
                cursor = conn.execute(cmd)
                # cur.execute("SELECT * FROM EMP WHERE emp_id=" + str(emp_id))
                rows = cursor.fetchall()
#                 print(rows)
                profile = None
                for row in rows :
#                     print("ID = ", row[0])
                    profile = row
                conn.close()
                return profile

In [4]:
getProfile("110102")

--------------------------------------------------------------------------- 
 OperationalError Traceback (most recent call last)
 <command-2920171247669905> in <module> 
 ----> 1 getProfile ( "110102" ) 

 <command-4470180574024725> in getProfile (emp_id) 
 1 def getProfile ( emp_id ) : 
 ----> 2 conn = sqlite3 . connect ( "../../dbfs/FileStore/tables/FR/dev.db" ) 
 3 # cur = conn.cursor() 
 4 cmd = "SELECT * FROM EMP WHERE emp_id=" + str ( emp_id ) 
 5 cursor = conn . execute ( cmd ) 

 OperationalError : unable to open database file

In [5]:
from pykafka import KafkaClient
from pykafka.common import OffsetType
import io
import numpy as np
from matplotlib import pyplot as plt
def get_kafka_client():
    return KafkaClient(hosts='2.tcp.ngrok.io:12218')


client = get_kafka_client()

for i in client.topics['images'].get_simple_consumer(auto_offset_reset=OffsetType.LATEST,reset_offset_on_start=True): 
# for i in client.topics['images'].get_simple_consumer(): 
    img=np.frombuffer(i.value,dtype='uint8')
    img=img.reshape(100,100)
    profile=recognizer(img)
    send_profile_data(str(profile).encode())
    

--------------------------------------------------------------------------- 
 SocketDisconnectedError Traceback (most recent call last)
 /databricks/python/lib/python3.7/site-packages/pykafka/simpleconsumer.py in fetch (self) 
 803 timeout = self . _fetch_wait_max_ms , 
 --> 804 min_bytes = self . _fetch_min_bytes
 805 )

 /databricks/python/lib/python3.7/site-packages/pykafka/broker.py in wrapped (self, *args, **kwargs) 
 44 raise SocketDisconnectedError
 ---> 45 return fn ( self , * args , ** kwargs ) 
 46 return wrapped

 /databricks/python/lib/python3.7/site-packages/pykafka/broker.py in fetch_messages (self, partition_requests, timeout, min_bytes) 
 326 # XXX - this call returns even with less than min_bytes of messages? 
 --> 327 return future . get ( response_class , broker_version = self . _broker_version ) 
 328 

 /databricks/python/lib/python3.7/site-packages/pykafka/handlers.py in get (self, response_cls, timeout, **response_kwargs) 
 73 if self . error : 
 ---> 74 raise self . error
 75 if response_cls : 

 /databricks/python/lib/python3.7/site-packages/pykafka/handlers.py in worker () 
 213 if task . future : 
 --> 214 res = shared . connection . response ( ) 
 215 task . future . set_response ( res ) 

 /databricks/python/lib/python3.7/site-packages/pykafka/connection.py in response (self) 
 225 self . disconnect ( ) 
 --> 226 raise SocketDisconnectedError ( "<broker {}:{}>" . format ( self . host , self . port ) ) 
 227 size += r

 SocketDisconnectedError : <broker b'2.tcp.ngrok.io':12218>

During handling of the above exception, another exception occurred:

 NoBrokersAvailableError Traceback (most recent call last)
 <command-2920171247669901> in <module> 
 10 client = get_kafka_client ( ) 
 11 
 ---> 12 for i in client . topics [ 'images' ] . get_simple_consumer ( auto_offset_reset = OffsetType . LATEST , reset_offset_on_start = True ) : 
 13 # for i in client.topics['images'].get_simple_consumer(): 
 14 img = np . frombuffer ( i . value , dtype = 'uint8' ) 

 /databricks/python/lib/python3.7/site-packages/pykafka/simpleconsumer.py in __iter__ (self) 
 459 """Yield an infinite stream of messages until the consumer times out""" 
 460 while True : 
 --> 461 message = self . consume ( block = True ) 
 462 if not message : 
 463 return 

 /databricks/python/lib/python3.7/site-packages/pykafka/simpleconsumer.py in consume (self, block, unblock_event) 
 481 ret = None 
 482 while True : 
 --> 483 self . _raise_worker_exceptions ( ) 
 484 self . _cluster . handler . sleep ( ) 
 485 if self . _messages_arrived . acquire ( blocking = block , timeout = timeout ) : 

 /databricks/python/lib/python3.7/site-packages/pykafka/simpleconsumer.py in _raise_worker_exceptions (self) 
 274 """Raises exceptions encountered on worker threads""" 
 275 if self . _worker_exception is not None : 
 --> 276 reraise ( * self . _worker_exception ) 
 277 
 278 def _update ( self ) : 

 /databricks/python/lib/python3.7/site-packages/six.py in reraise (tp, value, tb) 
 691 if value . __traceback__ is not tb : 
 692 raise value . with_traceback ( tb ) 
 --> 693 raise value
 694 finally : 
 695 value = None 

 /databricks/python/lib/python3.7/site-packages/pykafka/simpleconsumer.py in fetcher () 
 438 if not self . _running : 
 439 break 
 --> 440 self . fetch ( ) 
 441 self . _cluster . handler . sleep ( .01 ) 
 442 except ReferenceError : 

 /databricks/python/lib/python3.7/site-packages/pykafka/simpleconsumer.py in fetch (self) 
 809 log.info("Updating cluster in response to error in fetch() "
 810 "for broker id %s", broker.id)
 --> 811 self . _update ( ) 
 812 # If the broker dies while we're supposed to stop, 
 813 # it's fine, and probably an integration test. 

 /databricks/python/lib/python3.7/site-packages/pykafka/simpleconsumer.py in _update (self) 
 280 # only allow one thread to be updating the consumer at a time 
 281 with self . _update_lock : 
 --> 282 self . _cluster . update ( ) 
 283 self . _setup_partitions_by_leader ( ) 
 28

In [6]:
rec = cv2.face.LBPHFaceRecognizer_create()
rec.read('../../dbfs/FileStore/tables/FR/trainer.yml')
def recognizer(img_data) :
            profile="unknown"
            emp_id, conf = rec.predict(img_data)
            if(conf>0.55):
              profile = getProfile(emp_id)
                
            print(profile,conf)
            return profile
